In [1]:
from skimage.exposure import rescale_intensity
import numpy as np
import random, os

import argparse
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import random
from matplotlib import pyplot as plt
import os, random
import numpy as np
from scipy import ndimage
from scipy import misc as sm
import cv2

In [1]:
def clamp(val, frm, to):
    if (val <= frm):
        return frm
    if (val >= to):
        return to
    return val

def apply_mat(image,i,j, mat_app):
    vr = vg = vb = 0
    clampi = i+len(mat_app)
    clampj = j+len(mat_app[0])
    
    if(clampi > len(image) or clampj > len(image[0])):
        return
    
    for c in range(i, clampi):
        for d in range(j, clampj):
            vr += image[c][d][0] * mat_app[c-i][d-j]
            vg += image[c][d][1] * mat_app[c-i][d-j]
            vb += image[c][d][2] * mat_app[c-i][d-j]
    image[i][j][0] = clamp(vr,0,255)        
    image[i][j][1] = clamp(vg,0,255)        
    image[i][j][2] = clamp(vb,0,255)
    
def apply_multichannel_mat(image, i, j, kernel_channels):
    vr = vg = vb = 0
    clampi = i+len(mat_app)
    clampj = j+len(mat_app[0])
    
    if(clampi > len(image) or clampj > len(image[0])):
        return
    
    for c in range(i, clampi):
        for d in range(j, clampj):
            vr += image[c][d][0] * kernel_channels[0][c-i][d-j]
            vg += image[c][d][1] * kernel_channels[1][c-i][d-j]
            vb += image[c][d][2] * kernel_channels[2][c-i][d-j]
    image[i][j][0] = clamp(vr,0,255)        
    image[i][j][1] = clamp(vg,0,255)        
    image[i][j][2] = clamp(vb,0,255)
    
    
def overlap(orig, other, other_opacity):
    output = np.copy(orig)
    for i in range(len(orig)):
        for j in range(len(orig[0])):
            output[i][j] = (1-other_opacity)*orig[i][j] + other_opacity*other[i][j]
    return output


def random_noise(orig, amount = 1, artifact_size = 0):
    output = np.copy(orig)
    for i in range(len(orig)):
        for j in range(len(orig[0])):
            if(np.random.rand() < amount):
                value = np.random.rand() *output[i][j] 
                output[i][j] = value
                for x in range(i-artifact_size,i + artifact_size):    
                    if (x > 0 and x < len(orig)):
                        output[x][j] = value
                        
                for y in range(j-artifact_size,j + artifact_size):    
                    if (y > 0 and y < len(orig[0])):
                        output[i][y] = value
    return output

def salt_and_pepper(orig, amount = 1):
    output = np.copy(orig)
    for i in range(len(orig)):
        for j in range(len(orig[0])):
            if(np.random.rand() < amount):
                if(np.random.rand() < 0.5):
                    output[i][j] = (0,0,0)
                else:
                    output[i][j] = (1,1,1)
    return output

def swap_pixels(orig, amount = 1):
    output = np.copy(orig)
    h = len(orig)
    w = len(orig[0])
    for i in range(len(orig)):
        for j in range(len(orig[0])):
            if(np.random.rand() < amount):
                defaultp = output[i][j]
                gi, gj = np.random.randint(0,h), np.random.randint(0,w)
                output[i][j] = output[gi][gj]
                output[gi][gj] = defaultp
                
    return output

def color_int(col):
    return max(min(round(col), 255), 0)

def boxed2flat(row):
    return [_color_int(x) for box in row for x in box]

def image2file(image, path):
    """ Writes an image in list of lists format to a file. Will work with
        either color or grayscale. """
    img = image
    with open(path, 'wb') as f:
        png.Writer(width=len(image[0]), height=len(image)).write(f, [boxed2flat(r) for r in img])
        
def recursive_files(dir):
    files = []
    for r, d, f in os.walk(dir):
        for file in f:
                files.append(os.path.join(r, file))
    return files
            
def online_choice(iterable):
    for n, x in enumerate(iterable, 1):
        if random.randrange(n) == 0:
            pick = x
    return pick

In [3]:
n=1000
random.seed();

rfiles = recursive_files("./lsun/church_dataset")
print(type(list(rfiles)))
for iteration in range(n):
    rfile = rfiles[iteration]
    
    print(f"[{iteration+1}/{n}] {rfile}")
    
    img = cv2.imread(rfile, cv2.IMREAD_COLOR)
    img = resize(img, (256, 256), anti_aliasing=True)
    orig = np.copy(img)

    random_noised  = salt_and_pepper(img, 0.03) 
    #random_noised  = random_noise(img,0.4,2)
    #random_noised  = swap_pixels(img, 0.55)
    
    together = overlap(orig, random_noised, 1)
    
    r_parts = rfile.split('\\')
    r_end = r_parts[len(r_parts)-1]
                              
    no_suffix = r_end.split('.')[0]
    
    cv2.imwrite(f"./tmp/{no_suffix}_orig.jpg", orig*256)
    cv2.imwrite(f"./tmp/{no_suffix}_noised.jpg", together*256)

<class 'list'>
[1/10] ./lsun/church_dataset/0000955125.jpg
[2/10] ./lsun/church_dataset/00018cf7eb.jpg
[3/10] ./lsun/church_dataset/0001b23b9c.jpg
[4/10] ./lsun/church_dataset/0002200c16.jpg
[5/10] ./lsun/church_dataset/000238fdcd.jpg
[6/10] ./lsun/church_dataset/00024cd7b6.jpg
[7/10] ./lsun/church_dataset/000281b0c6.jpg
[8/10] ./lsun/church_dataset/00029d64a9.jpg
[9/10] ./lsun/church_dataset/00036953d8.jpg
[10/10] ./lsun/church_dataset/0004f86fef.jpg
